In [2]:
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset as nc
import math
import statistics as stat
import pandas as pd
import xarray as xr
import csv

file1 = '/Users/kudou/Documents/Classes\Winter 2021/SeniorThesis/CODE/data/Leatherman_All/leathermanNavy_3d_nosink/release_2018.01.01.nc'
file2 = '/Users/kudou/Documents/Classes\Winter 2021/SeniorThesis/CODE/data/Leatherman_All/leathermanNavy_3d_sink40_sink/release_2018.01.01.nc'

In [3]:
# Read netCDF4 files
ns = nc(file1)
s = nc(file2)

pd_ns = xr.open_dataset(file1, decode_times=False)
pd_s = xr.open_dataset(file2, decode_times=False)

# Convert float64 to data files 

ns_reldepth = ns["cs"][:].data
ns_lat = ns["lat"][:].data
ns_lon = ns["lon"][:].data
ns_depth = ns["h"][:].data

s_reldepth = s["cs"][:].data
s_lat = s["lat"][:].data
s_lon = s["lon"][:].data
s_depth = s["h"][:].data

ns_depthB = ns_reldepth < -0.99
ns_ind = np.argmax(ns_depthB, axis = 0)
ns_ind[ns_ind==0] = 720
ns_ind = tuple(ns_ind)

s_depthB = s_reldepth < -0.99
s_ind = np.argmax(s_depthB, axis = 0)
s_ind[s_ind==0] = 720
s_ind = tuple(s_ind)

pd_ns = pd_ns.to_dataframe()
pd_s = pd_s.to_dataframe()

pd_ns = pd_ns.take([0,1], axis=1)
pd_s = pd_s.take([0,1], axis=1)

In [4]:
n, m = 1000, 720
ns_dist, s_dist = np.zeros((n,m)), np.zeros((n,m))
ns_sum_dist, s_sum_dist = np.zeros((n,1)), np.zeros((n,1))

In [ ]:
for i in range(n):
    for j in range(m):
        start_latt, end_latt = pd_ns.loc[(i,j),"lat"], pd_ns.loc[(i,j+1),"lat"]
        start_long, end_long = pd_ns.loc[(i,j),"lon"], pd_ns.loc[(i,j+1),"lon"]
        d_long, d_latt = (end_long - start_long)*math.pi/180, (end_latt - start_latt)*math.pi/180
        a = math.sin(d_latt/2)**2 + math.cos(start_latt) * math.cos(end_latt) * math.sin(d_long/2)**2
        c = 2 * math.asin(math.sqrt(a))
        ns_dist[i,j] = 6371 * c
        
for i in range(n):
    ns_sum_dist[i] = ns_dist[i,0:ns_ind[i]].sum()
print(sum(ns_sum_dist)/n)

In [5]:
for i in range(n):
    for j in range(m):
        start_latt, end_latt = pd_s.loc[(i,j),"lat"], pd_s.loc[(i,j+1),"lat"] 
        start_long, end_long = pd_s.loc[(i,j),"lon"], pd_s.loc[(i,j+1),"lon"]
        d_long, d_latt = (end_long - start_long)*math.pi/180, (end_latt - start_latt)*math.pi/180
        a = math.sin(d_latt/2)**2 + math.cos(start_latt) * math.cos(end_latt) * math.sin(d_long/2)**2
        c = 2 * math.asin(math.sqrt(a))
        s_dist[i,j] = 6371 * c

for i in range(n):
    s_sum_dist[i] = s_dist[i,0:s_ind[i]].sum()
print(sum(s_sum_dist)/n)


[16.75029698]


In [1]:
ns_sum_dist.to_csv("ns_janNavy_dist.csv")    
s_sum_dist.to_csv("s_janNavy_dist.csv")

NameError: name 'ns_sum_dist' is not defined